# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [36]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "schools_complete.csv"
student_data_to_load = "students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
sdc = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

sdc.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [37]:
totSchools = school_data['school_name'].count()
totStudents = student_data['student_name'].count()
totBudget = school_data['budget'].sum()
avgMath = student_data['math_score'].mean()
avgRead = student_data['reading_score'].mean()
passRate = (avgMath + avgRead)/2
passMath = (student_data.loc[student_data['math_score'] >= 70, "math_score"].count())/totStudents
passRead = (student_data.loc[student_data['reading_score'] >= 70, "reading_score"].count())/totStudents
sumTable = pd.DataFrame({
    'Total Schools': [totSchools], 
    'Total Students': [totStudents], 
    'Total Budget': [totBudget],
    'Average Math Score': [avgMath],
    'Average Reading Score':[avgRead],
    '% Passing Math': [passMath],
    '% Passing Reading': [passRead],
    'Passing Rate': [passRate]})
sumTable


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Passing Rate
0,15,39170,24649428,78.985371,81.87784,0.749809,0.858055,80.431606


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [38]:
#Create initial data frame
school = pd.DataFrame({
    'School':sdc['school_name'],
    'Student':sdc['student_name'],
    'School Type': sdc['type'],
    'Budget':sdc['budget'],
    'Math':sdc['math_score'],
    'Read': sdc['reading_score'],
})

group = school.groupby('School')
schoolmath = school.groupby('School').mean()
schoolread = school.groupby('School').mean()
rateP = (schoolmath['Math'] + schoolread['Read'])/2


typeS= group['School Type'].max()
numStudents = group['Student'].count()
psBudget = group['Budget'].max()/group['Student'].count()
budget=  group['Budget'].max()
math = group['Math'].mean()
read = group['Read'].mean()

passMath  = school.drop(school[school.Math <70].index).groupby('School').count()
passRead  = school.drop(school[school.Read <70].index).groupby('School').count()
percentMath = passMath['Math']/numStudents
percentRead = passRead['Read']/numStudents
topSchools = pd.DataFrame({
    'Type': typeS,
    'Budget': budget,
    'Number of Students': numStudents,
    'Budget per Student' : psBudget,
    'Average Math Score': math,
    'Average Reading Score': read,
    '% Passing Math': percentMath,
    '% Passing Reading': percentRead,
    'Passing Rate': rateP
}).sort_values(by= 'Passing Rate', ascending = False)
topSchools.head(5)

,Type,Budget,Number of Students,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Passing Rate
School,,,,,,,,,
Pena High School,Charter,585858,962,609.0,83.839917,84.044699,0.945946,0.959459,83.942308
Wright High School,Charter,1049400,1800,583.0,83.682222,83.955000,0.933333,0.966111,83.818611
Holden High School,Charter,248087,427,581.0,83.803279,83.814988,0.925059,0.962529,83.809133
Thomas High School,Charter,1043130,1635,638.0,83.418349,83.848930,0.932722,0.973089,83.633639
Wilson High School,Charter,1319574,2283,578.0,83.274201,83.989488,0.938677,0.965396,83.631844


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [39]:
botSchools = pd.DataFrame({
    'Type': typeS,
    'Budget': budget,
    'Number of Students': numStudents,
    'Budget per Student' : psBudget,
    'Average Math Score': math,
    'Average Reading Score': read,
    '% Passing Math': percentMath,
    '% Passing Reading': percentRead,
    'Passing Rate': rateP
}).sort_values(by= 'Passing Rate', ascending = True).head(5)
botSchools

,Type,Budget,Number of Students,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Passing Rate
School,,,,,,,,,
Rodriguez High School,District,2547363,3999,637.0,76.842711,80.744686,0.663666,0.802201,78.793698
Huang High School,District,1910635,2917,655.0,76.629414,81.182722,0.656839,0.813164,78.906068
Ford High School,District,1763916,2739,644.0,77.102592,80.746258,0.683096,0.792990,78.924425
Figueroa High School,District,1884411,2949,639.0,76.711767,81.158020,0.659885,0.807392,78.934893
Johnson High School,District,3094650,4761,650.0,77.072464,80.966394,0.660576,0.812224,79.019429


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [40]:
ninth = student_data.drop(student_data[student_data.grade != '9th'].index).groupby('school_name').mean()
tenth = student_data.drop(student_data[student_data.grade != '10th'].index).groupby('school_name').mean()
eleventh = student_data.drop(student_data[student_data.grade != '11th'].index).groupby('school_name').mean()
twelfth = student_data.drop(student_data[student_data.grade != '12th'].index).groupby('school_name').mean()
nineMath = ninth['math_score']
tenMath = ninth['math_score']
elevenMath = ninth['math_score']
twelveMath = twelfth['math_score']

byGradeMath = pd.DataFrame({
    '9th': nineMath,
    '10th': tenMath,
    '11th': elevenMath,
    '12th': twelveMath
})
byGradeMath.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,77.083676,77.083676,76.492218
Cabrera High School,83.094697,83.094697,83.094697,83.277487
Figueroa High School,76.403037,76.403037,76.403037,77.151369
Ford High School,77.361345,77.361345,77.361345,76.179963
Griffin High School,82.044010,82.044010,82.044010,83.356164


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [10]:
nineRead = ninth['reading_score']
tenRead = ninth['reading_score']
elevenRead = ninth['reading_score']
twelveRead = twelfth['reading_score']

byGradeRead = pd.DataFrame({
    '9th': nineRead,
    '10th': tenRead,
    '11th': elevenRead,
    '12th': twelveRead
})
byGradeRead.head()

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,81.303155,81.303155,80.912451
Cabrera High School,83.676136,83.676136,83.676136,84.287958
Figueroa High School,81.198598,81.198598,81.198598,81.384863
Ford High School,80.632653,80.632653,80.632653,80.662338
Griffin High School,83.369193,83.369193,83.369193,84.013699


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [31]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [32]:
spending = topSchools.reset_index()
spending.drop(columns = ['School','Type', 'Budget'], axis = 1, inplace = True)

spending['Spending Ranges (Per Student)'] = pd.cut(spending['Budget per Student'], spending_bins, labels=group_names)
spendingRange = spending.groupby('Spending Ranges (Per Student)')
spendingRange.mean()

,Number of Students,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Passing Rate
Spending Ranges (Per Student),,,,,,,
<$585,1592.000000,581.000000,83.455399,83.933814,0.934601,0.966109,83.694607
$585-615,1361.500000,604.500000,83.599686,83.885211,0.942309,0.959003,83.742449
$615-645,2961.000000,635.166667,79.079225,81.891436,0.756682,0.861066,80.485330
$645-675,4104.333333,652.333333,76.997210,81.027843,0.661648,0.811340,79.012526


## Scores by School Size

* Perform the same operations as above, based on school size.

In [33]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]


In [34]:
size = topSchools.reset_index()
size.drop(columns = ['School','Type', 'Budget'], axis = 1, inplace = True)

size['Size Range'] = pd.cut(size['Number of Students'], size_bins, labels=group_names2)
sizeRange = size.groupby('Size Range')
sizeRange.mean().head()


,Number of Students,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Passing Rate
Size Range,,,,,,,
Small (<1000),694.500,595.000,83.821598,83.929843,0.935502,0.960994,83.875721
Medium (1000-2000),1704.400,605.600,83.374684,83.864438,0.935997,0.967907,83.619561
Large (2000-5000),3657.375,635.375,77.746417,81.344493,0.699634,0.827666,79.545455


## Scores by School Type

* Perform the same operations as above, based on school type.

In [91]:

group = school.groupby('School Type')
schoolmath = group.mean()
schoolread = group.mean()
rateP = (schoolmath['Math'] + schoolread['Read'])/2



numStudents = group['Student'].count()

math = group['Math'].mean()
read = group['Read'].mean()

passMath  = school.drop(school[school.Math <70].index).groupby('School Type').count()
passRead  = school.drop(school[school.Read <70].index).groupby('School Type').count()
percentMath = passMath['Math']/numStudents
percentRead = passRead['Read']/numStudents
groupType = pd.DataFrame({
   
    'Average Math Score': math,
    'Average Reading Score': read,
    '% Passing Math': percentMath,
    '% Passing Reading': percentRead,
    'Passing Rate': rateP
})
groupType

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,0.937018,0.966459,83.654502
District,76.987026,80.962485,0.665184,0.809052,78.974755
